In [ ]:
import scipy.io
import numpy as np
from sklearn.metrics import auc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Global efficiency after random attacks
1. Pour chaque iteration (240), faire l’AUC (en prenant 30-100%) pour chaque patient; réduction de la matrice à 240X1X15
2. Moyenner les valeurs ainsi obtenues pour chaque patient pour arriver à 15x1.

In [ ]:
hc_data_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/attacks240/attack_HC/rnd_attack_degrees_HC_05-12-2020 16-06v4.mat'
st0_data_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/attacks240/attack_ST01/rnd_attack_degrees_ST01_05-12-2020 17-46v4.mat'
st1_data_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/attacks240/attack_ST02/rnd_attack_degrees_ST02_05-12-2020 20-16v4.mat'
st2_data_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/attacks240/attack_ST03/rnd_attack_degrees_ST03_05-13-2020 18-19v4.mat'

In [ ]:
# global eff after attack
outcome_var = 'GlobEff_bin_new'
minimum_connectivity_threshold = 0.3

For this variable, data is available for the 8 remaining threshold bins [0.3-1.0]

In [ ]:
hc_data_mat = scipy.io.loadmat(hc_data_path)
st0_data_mat = scipy.io.loadmat(st0_data_path)
st1_data_mat = scipy.io.loadmat(st1_data_path)
st2_data_mat = scipy.io.loadmat(st2_data_path)

In [ ]:
n_hc = len(hc_data_mat[outcome_var][0][0][0])
n_st = len(st0_data_mat[outcome_var][0][0][0])
n_bins = 11
n_rois = 240

In [ ]:
def parse_mat_file(data_mat, n_subj, n_rois, outcome_var):
    # parsing matlab matrix (roi, subj, bin) to obtain np array (subj, bin, roi)
    glob_eff_random_attack = np.moveaxis(np.squeeze([[np.vstack(data_mat[outcome_var][0][0][roi_idx][subj_idx])
                                 for subj_idx in range(n_subj)]
                                    for roi_idx in range(n_rois)]), 0, -1)
    return glob_eff_random_attack

In [ ]:
hc_glob_eff_random_attack = parse_mat_file(hc_data_mat, n_hc, n_rois, outcome_var)
st0_glob_eff_random_attack = parse_mat_file(st0_data_mat, n_st, n_rois, outcome_var)
st1_glob_eff_random_attack = parse_mat_file(st1_data_mat, n_st, n_rois, outcome_var)
st2_glob_eff_random_attack = parse_mat_file(st2_data_mat, n_st, n_rois, outcome_var)

In [ ]:
# use auc only over predefined area of thresholds
def custom_auc(values_over_thresholds):
    # only analyse thresholds above minimum_connectivity_threshold
    minimum_connectivity_threshold_index = int(minimum_connectivity_threshold*10)  # here thresholds start at bin0
    connectivity_thresholds = np.arange(minimum_connectivity_threshold, 1.1, 0.1)

    return auc(connectivity_thresholds,
                values_over_thresholds[minimum_connectivity_threshold_index:])

In [ ]:
# take integral AUC over [0.3-1.0] interval
hc_gEff_auc_random_attack = np.apply_along_axis(custom_auc, arr=hc_glob_eff_random_attack, axis=1)
st0_gEff_auc_random_attack = np.apply_along_axis(custom_auc, arr=st0_glob_eff_random_attack, axis=1)
st1_gEff_auc_random_attack = np.apply_along_axis(custom_auc, arr=st1_glob_eff_random_attack, axis=1)
st2_gEff_auc_random_attack = np.apply_along_axis(custom_auc, arr=st2_glob_eff_random_attack, axis=1)

In [ ]:
# take mean along attacked ROIs
hc_mean_gEff_auc_random_attack = np.mean(hc_gEff_auc_random_attack, axis=-1)
st0_mean_gEff_auc_random_attack = np.mean(st0_gEff_auc_random_attack, axis=-1)
st1_mean_gEff_auc_random_attack = np.mean(st1_gEff_auc_random_attack, axis=-1)
st2_mean_gEff_auc_random_attack = np.mean(st2_gEff_auc_random_attack, axis=-1)

In [ ]:
# convert to dataframe and subject column from index, then add info for group and timepoint
hc_mean_gEff_auc_random_attack_df = pd.DataFrame(hc_mean_gEff_auc_random_attack, columns=['mean_glob_eff_auc']).rename_axis('subject').reset_index()
hc_mean_gEff_auc_random_attack_df['group'] = 'hc'
hc_mean_gEff_auc_random_attack_df['timepoint'] = 0

st0_mean_gEff_auc_random_attack_df = pd.DataFrame(st0_mean_gEff_auc_random_attack, columns=['mean_glob_eff_auc']).rename_axis('subject').reset_index()
st0_mean_gEff_auc_random_attack_df['subject'] += n_hc
st0_mean_gEff_auc_random_attack_df['group'] = 'st'
st0_mean_gEff_auc_random_attack_df['timepoint'] = 0

st1_mean_gEff_auc_random_attack_df = pd.DataFrame(st1_mean_gEff_auc_random_attack, columns=['mean_glob_eff_auc']).rename_axis('subject').reset_index()
st1_mean_gEff_auc_random_attack_df['subject'] += n_hc
st1_mean_gEff_auc_random_attack_df['group'] = 'st'
st1_mean_gEff_auc_random_attack_df['timepoint'] = 1

st2_mean_gEff_auc_random_attack_df = pd.DataFrame(st2_mean_gEff_auc_random_attack, columns=['mean_glob_eff_auc']).rename_axis('subject').reset_index()
st2_mean_gEff_auc_random_attack_df['subject'] += n_hc
st2_mean_gEff_auc_random_attack_df['group'] = 'st'
st2_mean_gEff_auc_random_attack_df['timepoint'] = 2

In [ ]:

all_mean_glob_eff_auc_df = pd.concat([hc_mean_gEff_auc_random_attack_df, st0_mean_gEff_auc_random_attack_df,
                                      st1_mean_gEff_auc_random_attack_df, st2_mean_gEff_auc_random_attack_df],
                                ignore_index=True)
all_mean_glob_eff_auc_df['attack_type'] = 'random'

# convert auc to numeric
all_mean_glob_eff_auc_df['mean_glob_eff_auc'] = all_mean_glob_eff_auc_df['mean_glob_eff_auc'].astype('float')

In [ ]:
ax = sns.catplot(x="timepoint", y="mean_glob_eff_auc", hue="group",
            kind='box', dodge=True,
            data=all_mean_glob_eff_auc_df, palette="Set3"
                )
ax.fig.suptitle('Mean Global efficiency AUC after random attack')
ax.fig.subplots_adjust(top=0.85)
plt.show()

In [ ]:
# fuse with already existant dataframe
all_mean_glob_eff_auc_all_attacks_df = pd.read_csv('mean_glob_eff_auc_after_attack_df.csv')
all_mean_glob_eff_auc_all_attacks_df = pd.concat([all_mean_glob_eff_auc_all_attacks_df, all_mean_glob_eff_auc_df],
                                                 ignore_index=True)
all_mean_glob_eff_auc_all_attacks_df.to_csv('mean_glob_eff_auc_after_attack_df.csv', index=False)

## Build dataframe whilst keeping density bins

In [ ]:

# take mean along attacked ROIs without AUCs
minimum_connectivity_threshold_index = int(minimum_connectivity_threshold*10)  # here thresholds start at bin0

hc_mean_gEff_random_attack = np.mean(hc_glob_eff_random_attack, axis=-1)[:, minimum_connectivity_threshold_index:]
st0_mean_gEff_random_attack = np.mean(st0_glob_eff_random_attack, axis=-1)[:, minimum_connectivity_threshold_index:]
st1_mean_gEff_random_attack = np.mean(st1_glob_eff_random_attack, axis=-1)[:, minimum_connectivity_threshold_index:]
st2_mean_gEff_random_attack = np.mean(st2_glob_eff_random_attack, axis=-1)[:, minimum_connectivity_threshold_index:]

In [ ]:
# create df without AUC
# convert to dataframe and subject column from index, then add info for group and timepoint
hc_mean_gEff_random_attack_df = pd.DataFrame(hc_mean_gEff_random_attack).rename_axis('subject').reset_index()\
    .melt(id_vars=['subject'], var_name='density_bin', value_name='mean_glob_eff')
hc_mean_gEff_random_attack_df['group'] = 'hc'
hc_mean_gEff_random_attack_df['timepoint'] = 0

st0_mean_gEff_random_attack_df = pd.DataFrame(st0_mean_gEff_random_attack).rename_axis('subject').reset_index()\
    .melt(id_vars=['subject'], var_name='density_bin', value_name='mean_glob_eff')
st0_mean_gEff_random_attack_df['subject'] += n_hc
st0_mean_gEff_random_attack_df['group'] = 'st'
st0_mean_gEff_random_attack_df['timepoint'] = 0

st1_mean_gEff_random_attack_df = pd.DataFrame(st1_mean_gEff_random_attack).rename_axis('subject').reset_index()\
    .melt(id_vars=['subject'], var_name='density_bin', value_name='mean_glob_eff')
st1_mean_gEff_random_attack_df['subject'] += n_hc
st1_mean_gEff_random_attack_df['group'] = 'st'
st1_mean_gEff_random_attack_df['timepoint'] = 1

st2_mean_gEff_random_attack_df = pd.DataFrame(st2_mean_gEff_random_attack).rename_axis('subject').reset_index()\
    .melt(id_vars=['subject'], var_name='density_bin', value_name='mean_glob_eff')
st2_mean_gEff_random_attack_df['subject'] += n_hc
st2_mean_gEff_random_attack_df['group'] = 'st'
st2_mean_gEff_random_attack_df['timepoint'] = 2


In [ ]:
all_mean_glob_eff_df = pd.concat([hc_mean_gEff_random_attack_df, st0_mean_gEff_random_attack_df,
                                      st1_mean_gEff_random_attack_df, st2_mean_gEff_random_attack_df],
                                ignore_index=True)
all_mean_glob_eff_df['attack_type'] = 'random'
all_mean_glob_eff_df['density_bin'] += 3 # correct for bins starting at 0.3


# convert auc to numeric
all_mean_glob_eff_df['mean_glob_eff'] = all_mean_glob_eff_df['mean_glob_eff'].astype('float')


In [ ]:
ax = sns.catplot(x="timepoint", y="mean_glob_eff", hue="group",
            kind='box', dodge=True,
            data=all_mean_glob_eff_df, palette="Set3"
                )
ax.fig.suptitle('Mean Global efficiency after random attack')
ax.fig.subplots_adjust(top=0.85)
plt.show()


In [ ]:
# fuse with already existant dataframe
all_mean_glob_eff_all_attacks_df = pd.read_csv('mean_glob_eff_after_attack_df.csv')
all_mean_glob_eff_all_attacks_df = pd.concat([all_mean_glob_eff_all_attacks_df, all_mean_glob_eff_df],
                                                 ignore_index=True)
all_mean_glob_eff_all_attacks_df.to_csv('mean_glob_eff_after_attack_df.csv', index=False)
